# Solutions for Python Classes

<img src="../img/robotignite_logo_text.png"/>

## Index: 
* <a href="#SolutionExercise3-2">Solution Exercise P1</a>

<p id="SolutionExercise3-2"></p>

## Solution Exercise P1

* You will need to modify the class so that now, BB-8 robot stops moving after the specified time in the message has passed.

Here you have the python file with the class **MoveBB8**, with the modifications:

<p style="background:#3B8F10;color:white;" id="move_bb8_py">**Python Program: move_bb8.py** </p>

In [ ]:
#!/usr/bin/env python

import rospy
from geometry_msgs.msg import Twist

class MoveBB8():
    
    def __init__(self):
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.cmd = Twist()
        self.ctrl_c = False
        self.rate = rospy.Rate(1)
        rospy.on_shutdown(self.shutdownhook)
        
    def publish_once_in_cmd_vel(self):
        """
        This is because publishing in topics sometimes fails the first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(self.cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
        
    def shutdownhook(self):
        # works better than the rospy.is_shutdown()
        self.stop_bb8()
        self.ctrl_c = True
        
    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        self.cmd.linear.x = 0.0
        self.cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel()

    def move_bb8(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        self.cmd.linear.x = linear_speed
        self.cmd.angular.z = angular_speed
        i = 0
        
        rospy.loginfo("Moving BB8!")
        while not self.ctrl_c and i <= moving_time:
            self.publish_once_in_cmd_vel()
            i = i+1
            self.rate.sleep()
            
        self.stop_bb8()
            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        movebb8_object.move_bb8()
    except rospy.ROSInterruptException:
        pass

<p style="background:#3B8F10;color:white;" id="move_bb8_py">END **Python Program: move_bb8.py** </p><br>

* You will also need to modify the Service Server code, so that you pass the specified duration to the class.

<p style="background:#3B8F10;color:white;">**Python Program:  bb8_move_circle_service_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from my_custom_srv_msg_pkg.srv import MyCustomServiceMessage, MyCustomServiceMessageResponse
from bb8_move_circle_class import MoveBB8

def my_callback(request):
    rospy.loginfo("The Service move_bb8_in_circle has been called")
    movebb8_object = MoveBB8()
    movebb8_object.move_bb8(request.duration)
    rospy.loginfo("Finished service move_bb8_in_circle")
    response = MyCustomServiceMessageResponse()
    response.success = True
    return response

rospy.init_node('service_move_bb8_in_circle_server') 
my_service = rospy.Service('/move_bb8_in_circle', MyCustomServiceMessage , my_callback)
rospy.loginfo("Service /move_bb8_in_circle Ready")
rospy.spin()

<p style="background:#3B8F10;color:white;">END **Python Program: bb8_move_circle_service_server.py** </p>

Once you have it , test it through a **call** in the WebConsole to the service **/move_bb8_in_circle**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
rosservice call /move_bb8_in_circle [TAB]+[TAB]

You should get BB8 moving in circles, and stopping after the specified time has passed.

<p id="SolutionExercise3-3"></p>